In [1]:
import pandas as pd
import requests
import json
from datetime import datetime
import statistics as stats

## Request json file from S3 bucket

In [2]:
response = requests.get('https://ucsd-final-project-data.s3-us-west-2.amazonaws.com/sentiment_finance.json')
data = json.loads(response.text)
type(data)

list

In [3]:
df = pd.DataFrame(data)
df['date'] = df['unix_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
df.head()

,datetime,favorite_count,follower_count,retweet_count,screen_name,sentiment_by_sentence,text,tweet_id,unix_timestamp,username,date
0,29-Jun-2019 23:07:14.000000,304,3192272,117,CNBC,"[Neutral, Neutral]",A trainee day trader in France made $11.6 mill...,1145106478694588416,1.561875e+09,CNBC,2019-06-29
1,29-Jun-2019 21:55:38.000000,206,3192272,86,CNBC,[Negative],Krispy Kreme opening huge store in Times Squar...,1145088459083845633,1.561871e+09,CNBC,2019-06-29
2,29-Jun-2019 21:44:47.000000,34,3192272,18,CNBC,[Negative],Trump says 'there will be no reduction in the ...,1145085729577545728,1.561870e+09,CNBC,2019-06-29
3,29-Jun-2019 19:59:40.000000,23,3192272,13,CNBC,[Negative],'It's a temporary time out': Trump and Xi agre...,1145059277188546560,1.561864e+09,CNBC,2019-06-29
4,29-Jun-2019 19:04:33.000000,23,3192272,11,CNBC,"[Neutral, Neutral]",The Trump economy is starting to look more and...,1145045407648686084,1.561860e+09,CNBC,2019-06-29


### Encode and quantify the sentiment of financial tweets

In [4]:
new_list = []
for i in df['sentiment_by_sentence']:
    new_list.extend(i)
    
# list(set(new_list))
sent_dict = dict(zip(list(set(new_list)), [-2, -1, 1, 0, 2]))
sent_dict

{'Positive': -2,
 'Verypositive': -1,
 'Negative': 1,
 'Verynegative': 0,
 'Neutral': 2}

In [5]:
def encode_sentiment(sent_list, sent_dict):
    encoded_sent = [sent_dict[x] for x in sent_list]
    return stats.mean(encoded_sent)

# encode_sentiment(df['sentiment_by_sentence'][0], sent_dict)

In [6]:
def aggregate_sentiment(number):
    if number == 0:
        sentiment = "Neutral"
    elif number <0:
        sentiment = "Negative"
    else:
        sentiment = "Positive"
    return sentiment

# aggregate_sentiment(df['sentiment_encoded'][200])

In [7]:
df['sentiment_encoded'] = df['sentiment_by_sentence'].apply(encode_sentiment, args = (sent_dict,))  
df['sentiment_label'] = df['sentiment_encoded'].apply(aggregate_sentiment)
df['sentment_amplified'] = df['sentiment_encoded']*df["retweet_count"]

In [8]:
df.head()

,datetime,favorite_count,follower_count,retweet_count,screen_name,sentiment_by_sentence,text,tweet_id,unix_timestamp,username,date,sentiment_encoded,sentiment_label,sentment_amplified
0,29-Jun-2019 23:07:14.000000,304,3192272,117,CNBC,"[Neutral, Neutral]",A trainee day trader in France made $11.6 mill...,1145106478694588416,1.561875e+09,CNBC,2019-06-29,2.0,Positive,234.0
1,29-Jun-2019 21:55:38.000000,206,3192272,86,CNBC,[Negative],Krispy Kreme opening huge store in Times Squar...,1145088459083845633,1.561871e+09,CNBC,2019-06-29,1.0,Positive,86.0
2,29-Jun-2019 21:44:47.000000,34,3192272,18,CNBC,[Negative],Trump says 'there will be no reduction in the ...,1145085729577545728,1.561870e+09,CNBC,2019-06-29,1.0,Positive,18.0
3,29-Jun-2019 19:59:40.000000,23,3192272,13,CNBC,[Negative],'It's a temporary time out': Trump and Xi agre...,1145059277188546560,1.561864e+09,CNBC,2019-06-29,1.0,Positive,13.0
4,29-Jun-2019 19:04:33.000000,23,3192272,11,CNBC,"[Neutral, Neutral]",The Trump economy is starting to look more and...,1145045407648686084,1.561860e+09,CNBC,2019-06-29,2.0,Positive,22.0


## Load and transform tweets from presidential candidates

In [9]:
with open('data/sentiment_president.json', 'r') as f:
    prez_tweets = json.loads(f.read())

In [10]:
df_prez = pd.DataFrame(prez_tweets)
df_prez['date'] = df_prez['unix_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
df_prez.head()

,datetime,favorite_count,follower_count,retweet_count,screen_name,sentiment_by_sentence,text,tweet_id,unix_timestamp,username,date
0,02-Jul-2019 00:19:39.000000,13571,9361328,2930,BernieSanders,"[Negative, Negative, Neutral, Positive]",Are you truly free if you:\n-Can't afford to s...,1145849480765423621,1.562052e+09,Bernie Sanders,2019-07-02
1,01-Jul-2019 20:57:52.000000,1834,9361328,478,BernieSanders,"[Negative, Negative, Neutral]",A few years ago a $15 minimum wage was seen as...,1145798700935122944,1.562040e+09,Bernie Sanders,2019-07-01
2,01-Jul-2019 20:09:29.000000,735,9361328,204,BernieSanders,"[Negative, Negative, Neutral]",The @uofcincy Medical Center was wrong to reta...,1145786522123329536,1.562037e+09,Bernie Sanders,2019-07-01
3,01-Jul-2019 19:15:45.000000,3845,9361328,1038,BernieSanders,"[Negative, Positive, Negative, Negative, Neutral]",Donald Trump promised during his campaign that...,1145773002472218624,1.562034e+09,Bernie Sanders,2019-07-01
4,01-Jul-2019 15:28:08.000000,2293,9361328,407,BernieSanders,"[Positive, Positive, Neutral]",Thank you @WorkingFamilies. I was proud to rec...,1145715717825806337,1.562020e+09,Bernie Sanders,2019-07-01


### Encode and quantify the sentiment of political tweets

In [11]:
df_prez['sentiment_encoded'] = df_prez['sentiment_by_sentence'].apply(encode_sentiment, args = (sent_dict,))  
df_prez['sentiment_label'] = df_prez['sentiment_encoded'].apply(aggregate_sentiment)
df_prez['sentment_amplified'] = df_prez['sentiment_encoded']*df_prez["retweet_count"]
df_prez.head()

,datetime,favorite_count,follower_count,retweet_count,screen_name,sentiment_by_sentence,text,tweet_id,unix_timestamp,username,date,sentiment_encoded,sentiment_label,sentment_amplified
0,02-Jul-2019 00:19:39.000000,13571,9361328,2930,BernieSanders,"[Negative, Negative, Neutral, Positive]",Are you truly free if you:\n-Can't afford to s...,1145849480765423621,1.562052e+09,Bernie Sanders,2019-07-02,0.500000,Positive,1465.000000
1,01-Jul-2019 20:57:52.000000,1834,9361328,478,BernieSanders,"[Negative, Negative, Neutral]",A few years ago a $15 minimum wage was seen as...,1145798700935122944,1.562040e+09,Bernie Sanders,2019-07-01,1.333333,Positive,637.333333
2,01-Jul-2019 20:09:29.000000,735,9361328,204,BernieSanders,"[Negative, Negative, Neutral]",The @uofcincy Medical Center was wrong to reta...,1145786522123329536,1.562037e+09,Bernie Sanders,2019-07-01,1.333333,Positive,272.000000
3,01-Jul-2019 19:15:45.000000,3845,9361328,1038,BernieSanders,"[Negative, Positive, Negative, Negative, Neutral]",Donald Trump promised during his campaign that...,1145773002472218624,1.562034e+09,Bernie Sanders,2019-07-01,0.600000,Positive,622.800000
4,01-Jul-2019 15:28:08.000000,2293,9361328,407,BernieSanders,"[Positive, Positive, Neutral]",Thank you @WorkingFamilies. I was proud to rec...,1145715717825806337,1.562020e+09,Bernie Sanders,2019-07-01,-0.666667,Negative,-271.333333


## Concatenate the two dataframes with tweets (df, df_prez)

In [12]:
df_sent = pd.concat([df,df_prez]).reset_index()

In [13]:
print(df.shape, df_prez.shape, df_sent.shape)
print(type(df_sent['date'][0]))

(8635, 14) (16119, 14) (24754, 15)
<class 'datetime.date'>


## Load S&P 500 data from csv file

In [14]:
df_stock = pd.read_csv('data/stock.csv')
df_stock.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078
1,2017-01-04,2272.820068,2261.600098,2261.600098,2270.750000,3764890000,2270.750000
2,2017-01-05,2271.500000,2260.449951,2268.179932,2269.000000,3761820000,2269.000000
3,2017-01-06,2282.100098,2264.060059,2271.139893,2276.979980,3339890000,2276.979980
4,2017-01-09,2275.489990,2268.899902,2273.590088,2268.899902,3217610000,2268.899902


In [15]:
df_stock['Change'] = df_stock['Close'] - df_stock['Open']
df_stock.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Change
0,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078,6.260010
1,2017-01-04,2272.820068,2261.600098,2261.600098,2270.750000,3764890000,2270.750000,9.149902
2,2017-01-05,2271.500000,2260.449951,2268.179932,2269.000000,3761820000,2269.000000,0.820068
3,2017-01-06,2282.100098,2264.060059,2271.139893,2276.979980,3339890000,2276.979980,5.840088
4,2017-01-09,2275.489990,2268.899902,2273.590088,2268.899902,3217610000,2268.899902,-4.690186


In [16]:
df_stock['Change_encoded'] = df_stock['Change'].apply(lambda x: 1 if x>0 else 0)
df_stock.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Change,Change_encoded
0,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078,6.260010,1
1,2017-01-04,2272.820068,2261.600098,2261.600098,2270.750000,3764890000,2270.750000,9.149902,1
2,2017-01-05,2271.500000,2260.449951,2268.179932,2269.000000,3761820000,2269.000000,0.820068,1
3,2017-01-06,2282.100098,2264.060059,2271.139893,2276.979980,3339890000,2276.979980,5.840088,1
4,2017-01-09,2275.489990,2268.899902,2273.590088,2268.899902,3217610000,2268.899902,-4.690186,0


In [17]:
df_stock['date_obj'] = df_stock['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
df_stock.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Change,Change_encoded,date_obj
0,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078,6.260010,1,2017-01-03
1,2017-01-04,2272.820068,2261.600098,2261.600098,2270.750000,3764890000,2270.750000,9.149902,1,2017-01-04
2,2017-01-05,2271.500000,2260.449951,2268.179932,2269.000000,3761820000,2269.000000,0.820068,1,2017-01-05
3,2017-01-06,2282.100098,2264.060059,2271.139893,2276.979980,3339890000,2276.979980,5.840088,1,2017-01-06
4,2017-01-09,2275.489990,2268.899902,2273.590088,2268.899902,3217610000,2268.899902,-4.690186,0,2017-01-09


In [18]:
print(df_stock.shape)
print(type(df_stock['date_obj'][0]))

(630, 10)
<class 'datetime.date'>


In [19]:
# print(df_sent['date'], df_stock['Date'])
df_sent.head()

,index,datetime,favorite_count,follower_count,retweet_count,screen_name,sentiment_by_sentence,text,tweet_id,unix_timestamp,username,date,sentiment_encoded,sentiment_label,sentment_amplified
0,0,29-Jun-2019 23:07:14.000000,304,3192272,117,CNBC,"[Neutral, Neutral]",A trainee day trader in France made $11.6 mill...,1145106478694588416,1.561875e+09,CNBC,2019-06-29,2.0,Positive,234.0
1,1,29-Jun-2019 21:55:38.000000,206,3192272,86,CNBC,[Negative],Krispy Kreme opening huge store in Times Squar...,1145088459083845633,1.561871e+09,CNBC,2019-06-29,1.0,Positive,86.0
2,2,29-Jun-2019 21:44:47.000000,34,3192272,18,CNBC,[Negative],Trump says 'there will be no reduction in the ...,1145085729577545728,1.561870e+09,CNBC,2019-06-29,1.0,Positive,18.0
3,3,29-Jun-2019 19:59:40.000000,23,3192272,13,CNBC,[Negative],'It's a temporary time out': Trump and Xi agre...,1145059277188546560,1.561864e+09,CNBC,2019-06-29,1.0,Positive,13.0
4,4,29-Jun-2019 19:04:33.000000,23,3192272,11,CNBC,"[Neutral, Neutral]",The Trump economy is starting to look more and...,1145045407648686084,1.561860e+09,CNBC,2019-06-29,2.0,Positive,22.0


In [20]:
df_merged = pd.merge(df_stock, df_sent, how = "inner", left_on = 'date_obj', right_on = 'date')
df_merged.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Change,Change_encoded,date_obj,...,screen_name,sentiment_by_sentence,text,tweet_id,unix_timestamp,username,date,sentiment_encoded,sentiment_label,sentment_amplified
0,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078,6.26001,1,2017-01-03,...,SenWarren,"[Neutral, Neutral]",Welcome @SenDuckworth &amp; @SenCortezMasto to...,816416593102114816,1.483513e+09,Elizabeth Warren,2017-01-03,2.0,Positive,2428.0
1,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078,6.26001,1,2017-01-03,...,SenWarren,[Negative],"Tell us, @GOP: Who, exactly, thinks that the p...",816118394680393728,1.483442e+09,Elizabeth Warren,2017-01-03,1.0,Positive,24793.0
2,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078,6.26001,1,2017-01-03,...,SenWarren,[Negative],And now the @HouseGOP is gutting its ethics of...,816118058603474945,1.483441e+09,Elizabeth Warren,2017-01-03,1.0,Positive,7677.0
3,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078,6.26001,1,2017-01-03,...,SenWarren,[Negative],Trump’s cabinet of billionaires &amp; CEOs hav...,816117758345826304,1.483441e+09,Elizabeth Warren,2017-01-03,1.0,Positive,4955.0
4,2017-01-03,2263.879883,2245.129883,2251.570068,2257.830078,3770530000,2257.830078,6.26001,1,2017-01-03,...,SenWarren,[Negative],.@realDonaldTrump still hasn’t announced plans...,816117476937371648,1.483441e+09,Elizabeth Warren,2017-01-03,1.0,Positive,2964.0


In [26]:
df_merged['date'] = df_merged['date'].apply(lambda x: x.strftime("%d-%m-%Y"))
df_merged['date'].dtype

dtype('O')

In [30]:
df_merged = df_merged.drop(columns = ['Date', 'date_obj'])

In [34]:
print(df_merged.shape, df.shape, df_prez.shape)

(19493, 23) (8635, 14) (16119, 14)


In [37]:
# check for any duplicate tweets
print(any(df_merged.duplicated(subset = "tweet_id")))

False


In [31]:
merged_dict = df_merged.to_dict(orient = 'records')
merged_dict[0]

{'High': 2263.8798828125,
 'Low': 2245.1298828125,
 'Open': 2251.570068359375,
 'Close': 2257.830078125,
 'Volume': 3770530000,
 'Adj Close': 2257.830078125,
 'Change': 6.260009765625,
 'Change_encoded': 1,
 'index': 14871,
 'datetime': '03-Jan-2017 22:50:58.000000',
 'favorite_count': 7359,
 'follower_count': 5125343,
 'retweet_count': 1214,
 'screen_name': 'SenWarren',
 'sentiment_by_sentence': ['Neutral', 'Neutral'],
 'text': 'Welcome @SenDuckworth &amp; @SenCortezMasto to the US Senate! https://t.co/cwWVPOWP0K',
 'tweet_id': 816416593102114816,
 'unix_timestamp': 1483512658.0,
 'username': 'Elizabeth Warren',
 'date': '03-01-2017',
 'sentiment_encoded': 2.0,
 'sentiment_label': 'Positive',
 'sentment_amplified': 2428.0}

In [32]:
type(merged_dict)

list

## Save merged file as a json file

In [33]:
with open('data/merged_stock_sentiment.json', 'w') as outfile:  
    json.dump(merged_dict, outfile)